In [2]:
import nltk
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
text = open("news2.txt").read().replace("\n\n", " ").replace("\n", " ")

In [4]:
def tknize(txt):
    tokenize_sentences = []
    tokenize_sentence_raw = sent_tokenize(txt)
    for sentence in tokenize_sentence_raw:
        tokenize_words = []
        for word in word_tokenize(sentence):
            tokenize_words.append(word)
        tokenize_sentences.append(tokenize_words)
    return tokenize_sentences, tokenize_sentence_raw

def removeStopWords(tokenize_text):
    englishStopwords = set(nltk.corpus.stopwords.words("english"))
    tknize_sentence = []
    for sentence in tokenize_text:
        tknize_words = []
        for word in sentence:
            if word not in englishStopwords: tknize_words.append(word)
        tknize_sentence.append(tknize_words)
    return tknize_sentence

def removePunch(tokenize_text):
    tokenize_sentence = []
    hash = set("""!@#$%^&*(),./<<>?;"';:}{`~\|[]""")
    for sentence in tokenize_text:
        tokenize_words = []
        for word in sentence:
            wrd = ""
            for chr in word:
                if chr not in hash:
                    wrd+=chr
            if len(wrd) > 0:
                tokenize_words.append(wrd)
        tokenize_sentence.append(tokenize_words)
    return tokenize_sentence

In [5]:
tokenize_sentences, tokenize_sentence_raw = tknize(text)
tokenize_text = removeStopWords(tokenize_sentences)
tokenize_text = removePunch(tokenize_text)

In [6]:
vec = Word2Vec(tokenize_text, min_count=1, vector_size=100, window=3, sg=1, epochs=100)

In [7]:
words = list(vec.wv.index_to_key)
print(words)


['movie', 'film', 'The', 'Maverick', 'Cruise', 's', 'happy', 'Gun', 'best', 'first', 'also', 'It', 'A', 'said', 'Top', 'showed', 'opening', 'ever', 'Tom', 'American', 'broken', 'Day', 'Memorial', 'Drive', 'record', 'broke', 'new', 'time', 'release', 'widest', 'Canada', 'theatres', 'USA', 'sales', 'cinemas', '4700', 'records', 'Over', 'biggest', 'history', 'number', 'North', 'He', 'Depp', 'beat', 'called', 'I', 'm', 'filmmakers', 'great', 'reviews', 'UK', 'newspaper', 'thrilling', 'over-the-top', 'blockbuster', 'Another', 'studio', 'action', 'years', 'At', 'want', 'fantastic', 'ridiculously', 'Johnny', 'weekend', 'make', '2007', 'Pirates', 'Caribbean', 'gave', '100', 'million', 'sequel', 'results', '1986', 'production', 'company', 'done', 'well', 'spokesman', 'These', 'My']


<h3>sentence vector</h3>

In [8]:
sentenceVector = []
for sentence in tokenize_text:
    sum = 0
    for word in sentence:
        sum+=vec.wv[word]
    sum/=len(sentence)
    sentenceVector.append(sum)


In [13]:
sentenceCosinSimilarity = {}
for i in range(len(sentenceVector)):
    for j in range(len(sentenceVector)):
        if i in sentenceCosinSimilarity:
            sentenceCosinSimilarity[i] += cosine_similarity([sentenceVector[i]], [sentenceVector[j]])[0][0]
        else: 
            sentenceCosinSimilarity[i] = cosine_similarity([sentenceVector[i]], [sentenceVector[j]])[0][0]
    sentenceCosinSimilarity[i]/=len(sentenceVector[i])
sentenceCosinSimilarity = [ [cosin, idx] for idx, cosin in sentenceCosinSimilarity.items() ]
sentenceCosinSimilarity.sort()

In [10]:
sim = [idx for sim, idx in sentenceCosinSimilarity[:5]]
sim.sort()
for idx in sim:
    print(tokenize_sentence_raw[idx])

It's the widest release of all time.
I'm happy for the filmmakers."
The movie has great reviews.
A UK newspaper called it a "thrilling" blockbuster.
He is happy he did.
